tgb - 11/13/2019 - Testing if rescaling the inputs changes the ability of the network to generalize.   
Architecture = From [], predict []   
Test 1 = Rephrase all water concentrations as RH  
Test 2 = Rephrase all temperature as deviations from moist adiabat  
  
tgb - 11/15/2019 - This notebook is mostly used for pre-processing

# 0) Imports

In [1]:
from cbrain.imports import *
from cbrain.data_generator import *
from cbrain.cam_constants import *
from cbrain.losses import *
from cbrain.utils import limit_mem
from cbrain.layers import *
import tensorflow as tf
from tensorflow import math as tfm
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
# physical_devices = tf.config.experimental.list_physical_devices('GPU') 
# tf.config.experimental.set_memory_growth(physical_devices[0], True)
# tf.config.experimental.set_memory_growth(physical_devices[1], True)
# tf.config.experimental.set_memory_growth(physical_devices[2], True)
import xarray as xr
import numpy as np
from cbrain.model_diagnostics import ModelDiagnostics
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as imag
import scipy.integrate as sin

TRAINDIR = '/local/Tom.Beucler/SPCAM_PHYS/'
DATADIR = '/project/meteo/w2w/A6/S.Rasp/SP-CAM/fluxbypass_aqua/'
PREFIX = '8col009_01_'



/nfspool-0/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog


# 1) Preprocessing

In [2]:
%cd /export/nfs0home/tbeucler/CBRAIN-CAM

/nfspool-0/home/tbeucler/CBRAIN-CAM


In [3]:
!python preprocessing-11132019.py -c pp_config/2022_07_30_Nando_O3.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
08/28/2022 02:04:18 AM Preprocess training dataset with real_geography flag set to False
08/28/2022 02:04:18 AM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2022_08_28_TRAIN_For_Nando_O3VMR.nc
08/28/2022 02:04:18 AM Reading input files
08/28/2022 02:04:18 AM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/Nando/SPCAM3.0_8col_03/SPCAM3.0_8col_03.cam2.h1.0000-0[5-8]-*-*.nc
/nfspool-0/home/tbeucler/CBRAIN-CAM/cbrain/preprocessing/convert_dataset_20191113.py:786: FutureWarning: In xarray version 0.15 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  ds = xr.open_mfdataset(in_fns, decode_times=False, decode_cf=False, concat_dim='time')
/DFS-L/DATA/pritchard/tbeucler/Miniconda3_2021_04_12/miniconda3/envs/CbrainCustomLayer/lib/python

In [4]:
!python preprocessing-11132019.py -c pp_config/2022_08_28_Nando_O3_Test.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
08/28/2022 02:36:35 AM Preprocess training dataset with real_geography flag set to False
08/28/2022 02:36:35 AM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2022_08_28_TEST_For_Nando_O3VMR.nc
08/28/2022 02:36:35 AM Reading input files
08/28/2022 02:36:35 AM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/Nando/SPCAM3.0_8col_03/SPCAM3.0_8col_03.cam2.h1.0001-0[6-9]-*-*.nc
/nfspool-0/home/tbeucler/CBRAIN-CAM/cbrain/preprocessing/convert_dataset_20191113.py:786: FutureWarning: In xarray version 0.15 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  ds = xr.open_mfdataset(in_fns, decode_times=False, decode_cf=False, concat_dim='time')
/DFS-L/DATA/pritchard/tbeucler/Miniconda3_2021_04_12/miniconda3/envs/CbrainCustomLayer/lib/python3

In [5]:
!python preprocessing-11132019.py -c pp_config/2022_08_28_Norm_For_Nando_O3VM.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
08/28/2022 02:59:39 AM Preprocess training dataset with real_geography flag set to False
08/28/2022 02:59:39 AM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2022_08_28_SMALL_For_Nando_03VMR.nc
08/28/2022 02:59:39 AM Reading input files
08/28/2022 02:59:39 AM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/Nando/SPCAM3.0_8col_03/SPCAM3.0_8col_03.cam2.h1.0001-01-*-*.nc
/nfspool-0/home/tbeucler/CBRAIN-CAM/cbrain/preprocessing/convert_dataset_20191113.py:786: FutureWarning: In xarray version 0.15 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  ds = xr.open_mfdataset(in_fns, decode_times=False, decode_cf=False, concat_dim='time')
/DFS-L/DATA/pritchard/tbeucler/Miniconda3_2021_04_12/miniconda3/envs/CbrainCustomLayer/lib/python3.6/

In [3]:
!python preprocessing-11132019.py -c pp_config/2022_08_03_Train_Valid_for_Jerry_RH_m4K.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
08/03/2022 02:11:41 PM Preprocess training dataset with real_geography flag set to False
08/03/2022 02:11:41 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2022_08_03_TRAIN_For_Jerry_RH_m4K.nc
08/03/2022 02:11:41 PM Reading input files
08/03/2022 02:11:41 PM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/sp8fbp_minus4k/sp8fbp_minus4k.cam2.h2.0001-0[1-4]-*-*.nc
/nfspool-0/home/tbeucler/CBRAIN-CAM/cbrain/preprocessing/convert_dataset_20191113.py:785: FutureWarning: In xarray version 0.15 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  ds = xr.open_mfdataset(in_fns, decode_times=False, decode_cf=False, concat_dim='time')
/DFS-L/DATA/pritchard/tbeucler/Miniconda3_2021_04_12/miniconda3/envs/CbrainCustomLayer/lib/python3.6/site-

In [3]:
!python preprocessing-11132019.py -c pp_config/2022_08_01_Train_Valid-M4K_Jerry.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
08/02/2022 12:51:10 PM Preprocess training dataset with real_geography flag set to False
08/02/2022 12:51:10 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2022_08_01_TRAIN_M4K_First_Eight_Months.nc
08/02/2022 12:51:10 PM Reading input files
08/02/2022 12:51:10 PM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/sp8fbp_minus4k/sp8fbp_minus4k.cam2.h2.0000-0[1-8]-*-*.nc
/nfspool-0/home/tbeucler/CBRAIN-CAM/cbrain/preprocessing/convert_dataset_20191113.py:785: FutureWarning: In xarray version 0.15 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  ds = xr.open_mfdataset(in_fns, decode_times=False, decode_cf=False, concat_dim='time')
/DFS-L/DATA/pritchard/tbeucler/Miniconda3_2021_04_12/miniconda3/envs/CbrainCustomLayer/lib/python3.6

In [3]:
!python preprocessing-11132019.py -c pp_config/2022_07_30_Nando_O3.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
07/30/2022 09:55:09 AM Preprocess training dataset with real_geography flag set to False
07/30/2022 09:55:09 AM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2022_07_30_TRAIN_For_Nando_O3VMR.nc
07/30/2022 09:55:09 AM Reading input files
07/30/2022 09:55:09 AM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/Nando/SPCAM3.0_8col_02/SPCAM3.0_8col_02.cam2.h1.0000-0[5-8]-*-*.nc
/nfspool-0/home/tbeucler/CBRAIN-CAM/cbrain/preprocessing/convert_dataset_20191113.py:785: FutureWarning: In xarray version 0.15 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  ds = xr.open_mfdataset(in_fns, decode_times=False, decode_cf=False, concat_dim='time')
/DFS-L/DATA/pritchard/tbeucler/Miniconda3_2021_04_12/miniconda3/envs/CbrainCustomLayer/lib/python

In [4]:
!python preprocessing-11132019.py -c pp_config/2021_03_18_O3_Train_Valid-P4K.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
04/13/2022 08:44:08 AM Preprocess training dataset with real_geography flag set to False
04/13/2022 08:44:08 AM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_03_18_O3_TRAIN_P4K.nc
04/13/2022 08:44:08 AM Reading input files
04/13/2022 08:44:08 AM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/sp8fbp_4k/sp8fbp_4k.cam2.h2.0001-0[1-4]-*-*.nc
04/13/2022 08:44:12 AM Crop levels
04/13/2022 08:44:12 AM Create stacked dataarray
04/13/2022 08:44:12 AM Real geography flag set to False
04/13/2022 08:44:12 AM Aquaplanet simulation detected
var is  QBP
var is  TBP
var is  O3_AQUA
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHFLX
var is  PHQ
var is  TPHYSTND
var is  QRL
var is  QRS
These time steps are cut: []
04/13/2022 08:44:41 AM Stack and reshape dataarray
04/13/2022 08:45:09 AM Save dataarray as /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_03_18_O3_TRAIN_P4K.nc
OMP: Info #212: KMP_AFFINITY: decoding x2APIC id

OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238199 thread 48 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238345 thread 49 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238346 thread 50 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238347 thread 51 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238348 thread 52 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238349 thread 53 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238351 thread 55 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238350 thread 54 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238352 thread 56 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238353 thread 57 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238354 thread 58 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238355 thread 

OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238207 thread 144 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238496 thread 146 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238495 thread 145 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238497 thread 147 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238498 thread 148 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238499 thread 149 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238500 thread 150 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238501 thread 151 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238502 thread 152 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238503 thread 153 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238505 thread 155 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238

OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238220 thread 240 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238605 thread 242 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238604 thread 241 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238606 thread 243 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238607 thread 244 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238608 thread 245 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238610 thread 247 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238609 thread 246 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238611 thread 248 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238613 thread 250 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238612 thread 249 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238

OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238237 thread 336 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238753 thread 338 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238752 thread 337 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238755 thread 340 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238754 thread 339 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238756 thread 341 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238757 thread 342 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238759 thread 344 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238758 thread 343 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238762 thread 347 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238760 thread 345 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238

OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238253 thread 432 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238861 thread 434 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238860 thread 433 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238863 thread 436 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238862 thread 435 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238864 thread 437 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238865 thread 438 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238867 thread 440 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238866 thread 439 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238868 thread 441 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238869 thread 442 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238

OMP: Info #250: KMP_AFFINITY: pid 237702 tid 238264 thread 528 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 239012 thread 530 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 239011 thread 529 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 239014 thread 532 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 239013 thread 531 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 239015 thread 533 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 239016 thread 534 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 239017 thread 535 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 239018 thread 536 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 239019 thread 537 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 239021 thread 539 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 237702 tid 239

In [4]:
!python preprocessing-11132019.py -c pp_config/2022_02_16_Train_Valid_for_Nando_p4K.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
02/16/2022 02:17:29 PM Preprocess training dataset with real_geography flag set to False
02/16/2022 02:17:29 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2022_02_16_TRAIN_For_Nando_p4K.nc
02/16/2022 02:17:29 PM Reading input files
02/16/2022 02:17:29 PM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/sp8fbp_4k/sp8fbp_4k.cam2.h2.0001-0[1-4]-*-*.nc
02/16/2022 02:20:42 PM Crop levels
02/16/2022 02:20:42 PM Create stacked dataarray
02/16/2022 02:20:42 PM Real geography flag set to False
02/16/2022 02:20:42 PM Aquaplanet simulation detected
var is  QBP
var is  TBP
var is  VBP
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHFLX
var is  PHQ
var is  TPHYSTND
var is  FSNT
var is  FSNS
var is  FLNT
var is  FLNS
var is  PRECT
These time steps are cut: []
02/16/2022 02:20:42 PM Stack and reshape dataarray
02/16/2022 02:21:14 PM Save dataarray as /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2022_02_16_TRAIN_For_Nando_p4K

OMP: Info #250: KMP_AFFINITY: pid 192604 tid 195567 thread 48 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 195667 thread 50 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 195666 thread 49 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 195668 thread 51 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 195669 thread 52 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 195671 thread 54 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 195670 thread 53 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 195672 thread 55 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 195673 thread 56 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 195674 thread 57 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 195675 thread 58 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 195676 thread 

OMP: Info #250: KMP_AFFINITY: pid 192604 tid 195577 thread 147 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 195834 thread 149 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 195833 thread 148 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 195579 thread 150 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 195839 thread 152 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 195838 thread 151 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 195841 thread 154 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 195840 thread 153 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 195843 thread 156 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 195842 thread 155 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 195844 thread 157 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 19

OMP: Info #250: KMP_AFFINITY: pid 192604 tid 195589 thread 249 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 196001 thread 250 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 196002 thread 251 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 196003 thread 252 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 196004 thread 253 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 196005 thread 254 bound to OS proc set 9
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 196006 thread 255 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 196007 thread 256 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 196009 thread 258 bound to OS proc set 13
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 196008 thread 257 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 196010 thread 259 bound to OS proc set 17
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 196

OMP: Info #250: KMP_AFFINITY: pid 192604 tid 195599 thread 348 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 196275 thread 349 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 196277 thread 351 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 196278 thread 352 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 196281 thread 355 bound to OS proc set 17
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 196280 thread 354 bound to OS proc set 13
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 196279 thread 353 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 196276 thread 350 bound to OS proc set 9
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 196283 thread 357 bound to OS proc set 23
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 196282 thread 356 bound to OS proc set 21
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 196285 thread 359 bound to OS proc set 15
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 196

OMP: Info #250: KMP_AFFINITY: pid 192604 tid 195609 thread 447 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 196396 thread 448 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 196397 thread 449 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 195568 thread 450 bound to OS proc set 13
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 196476 thread 451 bound to OS proc set 17
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 196477 thread 452 bound to OS proc set 21
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 196478 thread 453 bound to OS proc set 23
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 196479 thread 454 bound to OS proc set 19
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 196480 thread 455 bound to OS proc set 15
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 196481 thread 456 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 196482 thread 457 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 

OMP: Info #250: KMP_AFFINITY: pid 192604 tid 195566 thread 552 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 197712 thread 554 bound to OS proc set 32
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 197711 thread 553 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 197714 thread 556 bound to OS proc set 30
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 197713 thread 555 bound to OS proc set 34
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 197715 thread 557 bound to OS proc set 26
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 197716 thread 558 bound to OS proc set 36
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 197718 thread 560 bound to OS proc set 44
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 197717 thread 559 bound to OS proc set 40
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 197719 thread 561 bound to OS proc set 46
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 197720 thread 562 bound to OS proc set 42
OMP: Info #250: KMP_AFFINITY: pid 192604 ti

OMP: Info #250: KMP_AFFINITY: pid 192604 tid 195564 thread 636 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 208286 thread 637 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 208287 thread 638 bound to OS proc set 9
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 195590 thread 639 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 208289 thread 640 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 208290 thread 641 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 195606 thread 642 bound to OS proc set 13
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 208291 thread 643 bound to OS proc set 17
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 208294 thread 646 bound to OS proc set 19
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 208292 thread 644 bound to OS proc set 21
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 208293 thread 645 bound to OS proc set 23
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 208

OMP: Info #250: KMP_AFFINITY: pid 192604 tid 208469 thread 735 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 208470 thread 736 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 208471 thread 737 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 208472 thread 738 bound to OS proc set 13
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 208473 thread 739 bound to OS proc set 17
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 208474 thread 740 bound to OS proc set 21
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 208475 thread 741 bound to OS proc set 23
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 208476 thread 742 bound to OS proc set 19
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 208477 thread 743 bound to OS proc set 15
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 208478 thread 744 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 208479 thread 745 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 192604 tid 

02/16/2022 02:55:43 PM Done!
02/16/2022 02:55:46 PM Finish entire preprocessing script.


In [3]:
!python preprocessing-11132019.py -c pp_config/2022_02_16_Train_Valid_for_Nando_m4K.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
02/16/2022 01:37:12 PM Preprocess training dataset with real_geography flag set to False
02/16/2022 01:37:12 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2022_02_16_TRAIN_For_Nando_m4K.nc
02/16/2022 01:37:12 PM Reading input files
02/16/2022 01:37:12 PM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/sp8fbp_minus4k/sp8fbp_minus4k.cam2.h2.0001-0[1-4]-*-*.nc
02/16/2022 01:40:26 PM Crop levels
02/16/2022 01:40:26 PM Create stacked dataarray
02/16/2022 01:40:26 PM Real geography flag set to False
02/16/2022 01:40:26 PM Aquaplanet simulation detected
var is  QBP
var is  TBP
var is  VBP
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHFLX
var is  PHQ
var is  TPHYSTND
var is  FSNT
var is  FSNS
var is  FLNT
var is  FLNS
var is  PRECT
These time steps are cut: []
02/16/2022 01:40:27 PM Stack and reshape dataarray
02/16/2022 01:40:56 PM Save dataarray as /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2022_02_16_TRAIN_For

OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170094 thread 48 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170241 thread 49 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170242 thread 50 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170243 thread 51 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170244 thread 52 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170245 thread 53 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170246 thread 54 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170247 thread 55 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170248 thread 56 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170250 thread 58 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170249 thread 57 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170251 thread 

OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170104 thread 147 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170436 thread 148 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170437 thread 149 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170105 thread 150 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170452 thread 151 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170453 thread 152 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170454 thread 153 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170455 thread 154 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170456 thread 155 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170457 thread 156 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170458 thread 157 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 17

OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170116 thread 249 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170570 thread 251 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170569 thread 250 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170571 thread 252 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170572 thread 253 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170574 thread 255 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170573 thread 254 bound to OS proc set 9
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170576 thread 257 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170575 thread 256 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170577 thread 258 bound to OS proc set 13
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170579 thread 260 bound to OS proc set 21
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170

OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170126 thread 348 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170740 thread 349 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170741 thread 350 bound to OS proc set 9
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170742 thread 351 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170743 thread 352 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170744 thread 353 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170745 thread 354 bound to OS proc set 13
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170746 thread 355 bound to OS proc set 17
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170747 thread 356 bound to OS proc set 21
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170748 thread 357 bound to OS proc set 23
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170749 thread 358 bound to OS proc set 19
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170

OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170136 thread 447 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170894 thread 448 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170895 thread 449 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170095 thread 450 bound to OS proc set 13
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170924 thread 451 bound to OS proc set 17
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170925 thread 452 bound to OS proc set 21
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170926 thread 453 bound to OS proc set 23
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170928 thread 455 bound to OS proc set 15
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170927 thread 454 bound to OS proc set 19
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170930 thread 457 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170929 thread 456 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 

OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170135 thread 555 bound to OS proc set 34
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 171197 thread 556 bound to OS proc set 30
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 171198 thread 557 bound to OS proc set 26
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170089 thread 558 bound to OS proc set 36
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 171631 thread 559 bound to OS proc set 40
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 171632 thread 560 bound to OS proc set 44
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 172067 thread 561 bound to OS proc set 46
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 172068 thread 562 bound to OS proc set 42
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 172072 thread 566 bound to OS proc set 33
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 172069 thread 563 bound to OS proc set 38
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 172070 thread 564 bound to OS proc set 25
OMP: Info #250: KMP_AFFINITY: pid 167997 ti

OMP: Info #250: KMP_AFFINITY: pid 167997 tid 173760 thread 653 bound to OS proc set 26
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 173758 thread 651 bound to OS proc set 34
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 173763 thread 656 bound to OS proc set 44
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 173761 thread 654 bound to OS proc set 36
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 173759 thread 652 bound to OS proc set 30
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 173771 thread 664 bound to OS proc set 31
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 173772 thread 665 bound to OS proc set 27
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 173766 thread 659 bound to OS proc set 38
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 173768 thread 661 bound to OS proc set 29
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 173765 thread 658 bound to OS proc set 42
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 173764 thread 657 bound to OS proc set 46
OMP: Info #250: KMP_AFFINITY: pid 167997 ti

OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170115 thread 729 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 183754 thread 730 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 183755 thread 731 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 183756 thread 732 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 183757 thread 733 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 183758 thread 734 bound to OS proc set 9
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 183760 thread 736 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 183759 thread 735 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 183761 thread 737 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 183762 thread 738 bound to OS proc set 13
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 183763 thread 739 bound to OS proc set 17
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 183

OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170129 thread 825 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170099 thread 826 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 167997 tid 170133 thread 827 bound to OS proc set 14
02/16/2022 02:16:21 PM Done!
02/16/2022 02:16:23 PM Finish entire preprocessing script.


In [3]:
!python preprocessing-11132019.py -c pp_config/2022_02_07_Test_for_Nando_m4K

/nfspool-0/home/tbeucler/CBRAIN-CAM
02/07/2022 02:43:23 AM Preprocess training dataset with real_geography flag set to False
02/07/2022 02:43:23 AM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2022_02_07_TEST_For_Nando_m4K.nc
02/07/2022 02:43:23 AM Reading input files
02/07/2022 02:43:23 AM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/sp8fbp_minus4k/sp8fbp_minus4k.cam2.h2.0001-0[6-9]-*-*.nc
02/07/2022 02:46:58 AM Crop levels
02/07/2022 02:46:58 AM Create stacked dataarray
02/07/2022 02:46:58 AM Real geography flag set to False
02/07/2022 02:46:58 AM Aquaplanet simulation detected
var is  QBP
var is  TBP
var is  VBP
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHFLX
var is  PHQ
var is  TPHYSTND
var is  FSNT
var is  FSNS
var is  FLNT
var is  FLNS
var is  PRECT
These time steps are cut: []
02/07/2022 02:46:58 AM Stack and reshape dataarray
02/07/2022 02:47:30 AM Save dataarray as /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2022_02_07_TEST_For_N

OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4176 thread 48 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4292 thread 51 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4290 thread 49 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4291 thread 50 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4295 thread 54 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4294 thread 53 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4293 thread 52 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4296 thread 55 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4297 thread 56 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4298 thread 57 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4299 thread 58 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4300 thread 59 bound to OS proc set 14
OMP: Info #250: KMP_A

OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4188 thread 150 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4467 thread 152 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4466 thread 151 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4468 thread 153 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4469 thread 154 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4470 thread 155 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4471 thread 156 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4472 thread 157 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4473 thread 158 bound to OS proc set 9
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4475 thread 160 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4474 thread 159 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4476 thread 161 bound to OS proc set 3
OMP: Info

OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4198 thread 249 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4635 thread 250 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4637 thread 252 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4638 thread 253 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4636 thread 251 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4640 thread 255 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4639 thread 254 bound to OS proc set 9
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4641 thread 256 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4642 thread 257 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4643 thread 258 bound to OS proc set 13
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4644 thread 259 bound to OS proc set 17
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4645 thread 260 bound to OS proc set 21
OMP: Info

OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4208 thread 348 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4750 thread 349 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4751 thread 350 bound to OS proc set 9
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4752 thread 351 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4753 thread 352 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4754 thread 353 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4755 thread 354 bound to OS proc set 13
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4756 thread 355 bound to OS proc set 17
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4757 thread 356 bound to OS proc set 21
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4758 thread 357 bound to OS proc set 23
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4759 thread 358 bound to OS proc set 19
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4760 thread 359 bound to OS proc set 15
OMP: Info

OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4218 thread 447 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4937 thread 448 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4938 thread 449 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4173 thread 450 bound to OS proc set 13
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4940 thread 451 bound to OS proc set 17
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4941 thread 452 bound to OS proc set 21
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4942 thread 453 bound to OS proc set 23
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4943 thread 454 bound to OS proc set 19
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4944 thread 455 bound to OS proc set 15
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4945 thread 456 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4946 thread 457 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4948 thread 459 bound to OS proc set 34
OMP: I

OMP: Info #250: KMP_AFFINITY: pid 1717 tid 4175 thread 561 bound to OS proc set 46
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 11395 thread 562 bound to OS proc set 42
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 11396 thread 563 bound to OS proc set 38
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 11455 thread 565 bound to OS proc set 29
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 11454 thread 564 bound to OS proc set 25
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 11456 thread 566 bound to OS proc set 33
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 11457 thread 567 bound to OS proc set 35
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 11458 thread 568 bound to OS proc set 31
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 11460 thread 570 bound to OS proc set 37
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 11459 thread 569 bound to OS proc set 27
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 11461 thread 571 bound to OS proc set 41
OMP: Info #250: KMP_AFFINITY: pid 1717 tid 11462 thread 572 bound to OS proc 

In [4]:
!python preprocessing-11132019.py -c pp_config/2022_02_07_Test_for_Nando_p4K

/nfspool-0/home/tbeucler/CBRAIN-CAM
02/07/2022 03:12:58 AM Preprocess training dataset with real_geography flag set to False
02/07/2022 03:12:58 AM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2022_02_07_TEST_For_Nando_p4K.nc
02/07/2022 03:12:58 AM Reading input files
02/07/2022 03:12:58 AM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/sp8fbp_4k/sp8fbp_4k.cam2.h2.0001-0[6-9]-*-*.nc
02/07/2022 03:16:30 AM Crop levels
02/07/2022 03:16:30 AM Create stacked dataarray
02/07/2022 03:16:30 AM Real geography flag set to False
02/07/2022 03:16:30 AM Aquaplanet simulation detected
var is  QBP
var is  TBP
var is  VBP
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHFLX
var is  PHQ
var is  TPHYSTND
var is  FSNT
var is  FSNS
var is  FLNT
var is  FLNS
var is  PRECT
These time steps are cut: []
02/07/2022 03:16:30 AM Stack and reshape dataarray
02/07/2022 03:16:58 AM Save dataarray as /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2022_02_07_TEST_For_Nando_p4K.n

OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21142 thread 48 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21330 thread 50 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21329 thread 49 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21333 thread 53 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21332 thread 52 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21331 thread 51 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21334 thread 54 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21336 thread 56 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21335 thread 55 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21339 thread 59 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21338 thread 58 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21337 thread 57 bound to OS proc set 

OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21153 thread 147 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21429 thread 148 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21430 thread 149 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21155 thread 150 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21485 thread 152 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21484 thread 151 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21486 thread 153 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21487 thread 154 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21488 thread 155 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21489 thread 156 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21490 thread 157 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21491 thread 158 bound to

OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21165 thread 249 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21609 thread 250 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21610 thread 251 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21611 thread 252 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21612 thread 253 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21613 thread 254 bound to OS proc set 9
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21614 thread 255 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21615 thread 256 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21616 thread 257 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21617 thread 258 bound to OS proc set 13
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21618 thread 259 bound to OS proc set 17
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21619 thread 260 bound to 

OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21175 thread 348 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21786 thread 349 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21789 thread 352 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21787 thread 350 bound to OS proc set 9
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21788 thread 351 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21790 thread 353 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21791 thread 354 bound to OS proc set 13
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21792 thread 355 bound to OS proc set 17
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21794 thread 357 bound to OS proc set 23
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21793 thread 356 bound to OS proc set 21
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21795 thread 358 bound to OS proc set 19
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21796 thread 359 bound to 

OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21185 thread 447 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21954 thread 448 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21955 thread 449 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21143 thread 450 bound to OS proc set 13
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21975 thread 452 bound to OS proc set 21
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21974 thread 451 bound to OS proc set 17
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21976 thread 453 bound to OS proc set 23
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21977 thread 454 bound to OS proc set 19
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21978 thread 455 bound to OS proc set 15
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21980 thread 457 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21979 thread 456 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 21981 thread 458 bound 

OMP: Info #250: KMP_AFFINITY: pid 16917 tid 22972 thread 552 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 22973 thread 553 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 22974 thread 554 bound to OS proc set 32
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 22975 thread 555 bound to OS proc set 34
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 22976 thread 556 bound to OS proc set 30
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 22978 thread 558 bound to OS proc set 36
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 22977 thread 557 bound to OS proc set 26
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 22979 thread 559 bound to OS proc set 40
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 22981 thread 561 bound to OS proc set 46
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 22980 thread 560 bound to OS proc set 44
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 22983 thread 563 bound to OS proc set 38
OMP: Info #250: KMP_AFFINITY: pid 16917 tid 22984 thread 564 boun

In [3]:
!python preprocessing-11132019.py -c pp_config/2022_01_23_Train_Valid-RG-M4K-TPHYSTND500.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
01/23/2022 08:50:08 AM Preprocess training dataset with real_geography flag set to True
01/23/2022 08:50:08 AM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2022_01_23_TRAIN_RG_M4K_TPHYSTND500.nc
01/23/2022 08:50:08 AM Reading input files
01/23/2022 08:50:08 AM Reading input file /DFS-L/DATA/pritchard/liranp/To_Tom/Output/SST_4Km_Neuralnet_SPCAM_all_small_216/atm/hist/SST_4Km_Neuralnet_SPCAM_all_small_216.cam.h1.2009-*-0*-*.nc
/nfspool-0/home/tbeucler/CBRAIN-CAM/cbrain/preprocessing/convert_dataset_20191113.py:783: FutureWarning: In xarray version 0.15 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  ds = xr.open_mfdataset(in_fns, decode_times=False, decode_cf=False, concat_dim='time')
/DFS-L/DATA/pritchard/tbeucler/Miniconda3_2021_04

In [7]:
!python preprocessing-11132019.py -c pp_config/2022_01_19_Train_Valid-M4K-TPHYSTND500.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
01/19/2022 05:25:21 PM Preprocess training dataset with real_geography flag set to False
01/19/2022 05:25:21 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2022_01_19_TRAIN_M4K_TPHYSTND500.nc
01/19/2022 05:25:21 PM Reading input files
01/19/2022 05:25:21 PM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/sp8fbp_minus4k/sp8fbp_minus4k.cam2.h2.0001-0[1-4]-*-*.nc
01/19/2022 05:25:25 PM Crop levels
01/19/2022 05:25:25 PM Create stacked dataarray
01/19/2022 05:25:25 PM Real geography flag set to False
01/19/2022 05:25:25 PM Aquaplanet simulation detected
var is  QBP
var is  TBP
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHFLX
var is  TPHYSTND500
These time steps are cut: []
01/19/2022 05:25:25 PM Stack and reshape dataarray
01/19/2022 05:25:58 PM Save dataarray as /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2022_01_19_TRAIN_M4K_TPHYSTND500.nc
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KM

OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70192 thread 48 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70354 thread 50 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70353 thread 49 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70355 thread 51 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70356 thread 52 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70358 thread 54 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70357 thread 53 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70359 thread 55 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70360 thread 56 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70361 thread 57 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70362 thread 58 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70363 thread 59 bound to OS proc set 

OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70202 thread 168 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70533 thread 170 bound to OS proc set 32
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70532 thread 169 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70535 thread 172 bound to OS proc set 30
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70534 thread 171 bound to OS proc set 34
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70536 thread 173 bound to OS proc set 26
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70539 thread 176 bound to OS proc set 44
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70538 thread 175 bound to OS proc set 40
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70537 thread 174 bound to OS proc set 36
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70541 thread 178 bound to OS proc set 42
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70540 thread 177 bound to OS proc set 46
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70542 thread 179 boun

OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70214 thread 288 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70714 thread 289 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70716 thread 291 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70715 thread 290 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70718 thread 293 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70717 thread 292 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70721 thread 296 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70720 thread 295 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70719 thread 294 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70722 thread 297 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70723 thread 298 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70724 thread 299 bound to 

OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70224 thread 408 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70850 thread 409 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70851 thread 410 bound to OS proc set 32
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70852 thread 411 bound to OS proc set 34
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70854 thread 413 bound to OS proc set 26
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70853 thread 412 bound to OS proc set 30
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70856 thread 415 bound to OS proc set 40
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70857 thread 416 bound to OS proc set 44
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70855 thread 414 bound to OS proc set 36
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70860 thread 419 bound to OS proc set 38
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70859 thread 418 bound to OS proc set 42
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70858 thread 417 boun

OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70234 thread 528 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 71031 thread 530 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 71030 thread 529 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 71032 thread 531 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 71033 thread 532 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 71034 thread 533 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 71035 thread 534 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 71036 thread 535 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 71037 thread 536 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 71038 thread 537 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 71039 thread 538 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 71040 thread 539 bound to 

OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70193 thread 624 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 77965 thread 625 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 77966 thread 626 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 77967 thread 627 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 77968 thread 628 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 77969 thread 629 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 77971 thread 631 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 77972 thread 632 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 77970 thread 630 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 77974 thread 634 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 77973 thread 633 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 77976 thread 636 bound to 

OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70221 thread 744 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 78574 thread 747 bound to OS proc set 34
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 78573 thread 746 bound to OS proc set 32
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 78572 thread 745 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 78577 thread 750 bound to OS proc set 36
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 78575 thread 748 bound to OS proc set 30
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 78576 thread 749 bound to OS proc set 26
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 78578 thread 751 bound to OS proc set 40
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 78579 thread 752 bound to OS proc set 44
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 78580 thread 753 bound to OS proc set 46
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 78581 thread 754 bound to OS proc set 42
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 78582 thread 755 boun

OMP: Info #250: KMP_AFFINITY: pid 69898 tid 70189 thread 864 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 78778 thread 865 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 78779 thread 866 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 78781 thread 868 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 78780 thread 867 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 78783 thread 870 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 78782 thread 869 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 78784 thread 871 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 78788 thread 875 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 78787 thread 874 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 78786 thread 873 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 69898 tid 78785 thread 872 bound to 

In [4]:
!python preprocessing-11132019.py -c pp_config/2022_01_19_Train_Valid-P4K-TPHYSTND500.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
01/21/2022 01:02:48 PM Preprocess training dataset with real_geography flag set to False
01/21/2022 01:02:48 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2022_01_21_TRAIN_P4K_TPHYSTND500.nc
01/21/2022 01:02:48 PM Reading input files
01/21/2022 01:02:48 PM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/sp8fbp_4k/sp8fbp_4k.cam2.h2.0001-0[1-4]-*-*.nc
01/21/2022 01:03:00 PM Crop levels
01/21/2022 01:03:00 PM Create stacked dataarray
01/21/2022 01:03:00 PM Real geography flag set to False
01/21/2022 01:03:00 PM Aquaplanet simulation detected
var is  QBP
var is  TBP
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHFLX
var is  TPHYSTND500
These time steps are cut: []
01/21/2022 01:03:00 PM Stack and reshape dataarray
01/21/2022 01:03:53 PM Save dataarray as /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2022_01_21_TRAIN_P4K_TPHYSTND500.nc
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY

OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6123 thread 48 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6228 thread 50 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6227 thread 49 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6229 thread 51 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6230 thread 52 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6231 thread 53 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6232 thread 54 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6233 thread 55 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6235 thread 57 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6236 thread 58 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6234 thread 56 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6237 thread 59 bound to OS proc set 14
OMP: Info #250: KMP_A

OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6399 thread 160 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6398 thread 159 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6401 thread 162 bound to OS proc set 13
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6400 thread 161 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6403 thread 164 bound to OS proc set 21
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6402 thread 163 bound to OS proc set 17
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6404 thread 165 bound to OS proc set 23
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6408 thread 167 bound to OS proc set 15
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6407 thread 166 bound to OS proc set 19
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6133 thread 168 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6410 thread 170 bound to OS proc set 32
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6409 thread 169 bound to OS proc set 28
OMP: I

OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6141 thread 264 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6535 thread 266 bound to OS proc set 32
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6534 thread 265 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6536 thread 267 bound to OS proc set 34
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6537 thread 268 bound to OS proc set 30
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6538 thread 269 bound to OS proc set 26
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6539 thread 270 bound to OS proc set 36
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6540 thread 271 bound to OS proc set 40
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6541 thread 272 bound to OS proc set 44
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6542 thread 273 bound to OS proc set 46
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6543 thread 274 bound to OS proc set 42
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6544 thread 275 bound to OS proc set 38
OMP:

OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6155 thread 384 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6732 thread 386 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6733 thread 387 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6731 thread 385 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6734 thread 388 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6735 thread 389 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6736 thread 390 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6737 thread 391 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6738 thread 392 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6739 thread 393 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6740 thread 394 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6741 thread 395 bound to OS proc set 14
OMP: Info

...

OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6136 thread 624 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7051 thread 626 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7050 thread 625 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7052 thread 627 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7053 thread 628 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7054 thread 629 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7055 thread 630 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7056 thread 631 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7058 thread 633 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7057 thread 632 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7059 thread 634 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7060 thread 635 bound to OS proc set 14
OMP: Info

OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6146 thread 744 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7223 thread 745 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7224 thread 746 bound to OS proc set 32
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7225 thread 747 bound to OS proc set 34
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7226 thread 748 bound to OS proc set 30
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7227 thread 749 bound to OS proc set 26
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7228 thread 750 bound to OS proc set 36
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7229 thread 751 bound to OS proc set 40
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7231 thread 753 bound to OS proc set 46
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7230 thread 752 bound to OS proc set 44
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7232 thread 754 bound to OS proc set 42
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7233 thread 755 bound to OS proc set 38
OMP:

OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6163 thread 864 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7399 thread 865 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7400 thread 866 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7402 thread 868 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7401 thread 867 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7403 thread 869 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7404 thread 870 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7405 thread 871 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7406 thread 872 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7407 thread 873 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7408 thread 874 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7409 thread 875 bound to OS proc set 14
OMP: Info

OMP: Info #250: KMP_AFFINITY: pid 5528 tid 6124 thread 984 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7573 thread 985 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7574 thread 986 bound to OS proc set 32
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7575 thread 987 bound to OS proc set 34
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7576 thread 988 bound to OS proc set 30
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7577 thread 989 bound to OS proc set 26
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7578 thread 990 bound to OS proc set 36
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7579 thread 991 bound to OS proc set 40
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7580 thread 992 bound to OS proc set 44
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7581 thread 993 bound to OS proc set 46
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7582 thread 994 bound to OS proc set 42
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7583 thread 995 bound to OS proc set 38
OMP:

OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7971 thread 1088 bound to OS proc set 44
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7973 thread 1090 bound to OS proc set 42
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7972 thread 1089 bound to OS proc set 46
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 8049 thread 1093 bound to OS proc set 29
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7999 thread 1092 bound to OS proc set 25
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 7974 thread 1091 bound to OS proc set 38
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 8055 thread 1099 bound to OS proc set 41
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 8050 thread 1094 bound to OS proc set 33
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 8057 thread 1101 bound to OS proc set 47
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 8058 thread 1102 bound to OS proc set 43
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 8051 thread 1095 bound to OS proc set 35
OMP: Info #250: KMP_AFFINITY: pid 5528 tid 8056 thread 1100 bound to OS proc

In [4]:
!python preprocessing-11132019.py -c pp_config/2022_01_17_RG_Train_Valid-P4K_PRECTt-dt.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
01/18/2022 03:09:10 PM Preprocess training dataset with real_geography flag set to True
01/18/2022 03:09:10 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2022_01_17_RG_TRAIN_P4K_PRECTt-dt.nc
01/18/2022 03:09:10 PM Reading input files
01/18/2022 03:09:10 PM Reading input file /DFS-L/DATA/pritchard/liranp/To_Tom/Output/SST_4Kp_Neuralnet_SPCAM_all_v2_216/atm/hist/SST_4Kp_Neuralnet_SPCAM_all_v2_216.cam.h1.2009-*-0*-*.nc
01/18/2022 03:16:52 PM Crop levels
01/18/2022 03:16:52 PM Create stacked dataarray
01/18/2022 03:16:52 PM Real geography flag set to True
01/18/2022 03:16:52 PM Earth-like simulation detected
var is  QBP
var is  TBP
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHFLX
var is  PRECTt-dt
var is  PRECT
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0-47
OMP: Info #156: 

OMP: Info #250: KMP_AFFINITY: pid 103611 tid 110986 thread 36 bound to OS proc set 25
OMP: Info #250: KMP_AFFINITY: pid 103611 tid 110992 thread 37 bound to OS proc set 29
OMP: Info #250: KMP_AFFINITY: pid 103611 tid 110997 thread 38 bound to OS proc set 33
OMP: Info #250: KMP_AFFINITY: pid 103611 tid 110993 thread 39 bound to OS proc set 35
OMP: Info #250: KMP_AFFINITY: pid 103611 tid 111007 thread 40 bound to OS proc set 31
OMP: Info #250: KMP_AFFINITY: pid 103611 tid 111009 thread 41 bound to OS proc set 27
OMP: Info #250: KMP_AFFINITY: pid 103611 tid 111021 thread 42 bound to OS proc set 37
OMP: Info #250: KMP_AFFINITY: pid 103611 tid 111017 thread 43 bound to OS proc set 41
OMP: Info #250: KMP_AFFINITY: pid 103611 tid 111011 thread 44 bound to OS proc set 45
OMP: Info #250: KMP_AFFINITY: pid 103611 tid 110978 thread 45 bound to OS proc set 47
OMP: Info #250: KMP_AFFINITY: pid 103611 tid 110980 thread 46 bound to OS proc set 43
OMP: Info #250: KMP_AFFINITY: pid 103611 tid 110994 th

In [4]:
!python preprocessing-11132019.py -c pp_config/2022_01_13_Test_for_Nando_t-dt.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
01/13/2022 02:14:56 PM Preprocess training dataset with real_geography flag set to False
01/13/2022 02:14:56 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2022_01_13_TEST_For_Nando_t-dt.nc
01/13/2022 02:14:56 PM Reading input files
01/13/2022 02:14:56 PM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0000-0[6-9]-*-*.nc
01/13/2022 02:18:10 PM Crop levels
01/13/2022 02:18:10 PM Create stacked dataarray
01/13/2022 02:18:10 PM Real geography flag set to False
01/13/2022 02:18:10 PM Aquaplanet simulation detected
var is  QBP
var is  TBP
var is  VBP
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHFLX
var is  PHQ
var is  TPHYSTND
var is  FSNT
var is  FSNS
var is  FLNT
var is  FLNS
var is  PRECT
var is  PHQt-dt
var is  TPHYSTNDt-dt
var is  FSNTt-dt
var is  FSNSt-dt
var is  FLNTt-dt
var is  FLNSt-dt
var is  PRECTt-dt
These time steps are cut: []
01/13/2022 02:18:10 

OMP: Info #250: KMP_AFFINITY: pid 260063 tid 262605 thread 36 bound to OS proc set 25
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 262752 thread 37 bound to OS proc set 29
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 262753 thread 38 bound to OS proc set 33
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 262607 thread 39 bound to OS proc set 35
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 262755 thread 40 bound to OS proc set 31
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 262756 thread 41 bound to OS proc set 27
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 262609 thread 42 bound to OS proc set 37
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 262763 thread 44 bound to OS proc set 45
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 262762 thread 43 bound to OS proc set 41
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 262611 thread 45 bound to OS proc set 47
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 262769 thread 46 bound to OS proc set 43
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 262770 th

OMP: Info #250: KMP_AFFINITY: pid 260063 tid 263958 thread 139 bound to OS proc set 41
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 263957 thread 138 bound to OS proc set 37
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 263961 thread 142 bound to OS proc set 43
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 263960 thread 141 bound to OS proc set 47
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 263959 thread 140 bound to OS proc set 45
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 263963 thread 144 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 263964 thread 145 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 263962 thread 143 bound to OS proc set 39
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 263966 thread 147 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 263965 thread 146 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 263967 thread 148 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 26

OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264128 thread 243 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264129 thread 244 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264130 thread 245 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264132 thread 247 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264131 thread 246 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264133 thread 248 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264134 thread 249 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264135 thread 250 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264136 thread 251 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264137 thread 252 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264138 thread 253 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 26

OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264276 thread 348 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264277 thread 349 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264278 thread 350 bound to OS proc set 9
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264281 thread 353 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264280 thread 352 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264279 thread 351 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264282 thread 354 bound to OS proc set 13
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264283 thread 355 bound to OS proc set 17
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264284 thread 356 bound to OS proc set 21
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264286 thread 358 bound to OS proc set 19
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264285 thread 357 bound to OS proc set 23
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264

OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264456 thread 453 bound to OS proc set 23
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264457 thread 454 bound to OS proc set 19
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264459 thread 456 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264458 thread 455 bound to OS proc set 15
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264461 thread 458 bound to OS proc set 32
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264460 thread 457 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264462 thread 459 bound to OS proc set 34
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264463 thread 460 bound to OS proc set 30
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264464 thread 461 bound to OS proc set 26
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264465 thread 462 bound to OS proc set 36
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264466 thread 463 bound to OS proc set 40
OMP: Info #250: KMP_AFFINITY: pid 260063 ti

OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264904 thread 558 bound to OS proc set 36
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264905 thread 559 bound to OS proc set 40
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264907 thread 561 bound to OS proc set 46
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264906 thread 560 bound to OS proc set 44
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264908 thread 562 bound to OS proc set 42
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264909 thread 563 bound to OS proc set 38
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264910 thread 564 bound to OS proc set 25
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264911 thread 565 bound to OS proc set 29
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264913 thread 567 bound to OS proc set 35
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264914 thread 568 bound to OS proc set 31
OMP: Info #250: KMP_AFFINITY: pid 260063 tid 264912 thread 566 bound to OS proc set 33
OMP: Info #250: KMP_AFFINITY: pid 260063 ti

In [4]:
!python preprocessing-11132019.py -c pp_config/2022_01_10_Train_Valid_For_Nando_t-dt.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
01/10/2022 04:19:14 PM Preprocess training dataset with real_geography flag set to False
01/10/2022 04:19:14 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_09_02_TRAIN_For_Nando.nc
01/10/2022 04:19:14 PM Reading input files
01/10/2022 04:19:14 PM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0001-0[1-4]-*-*.nc
01/10/2022 04:19:17 PM Crop levels
01/10/2022 04:19:17 PM Create stacked dataarray
01/10/2022 04:19:17 PM Real geography flag set to False
01/10/2022 04:19:17 PM Aquaplanet simulation detected
var is  QBP
var is  TBP
var is  VBP
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHFLX
var is  PHQ
var is  TPHYSTND
var is  FSNT
var is  FSNS
var is  FLNT
var is  FLNS
var is  PRECT
var is  PHQt-dt
var is  TPHYSTNDt-dt
var is  FSNTt-dt
var is  FSNSt-dt
var is  FLNTt-dt
var is  FLNSt-dt
var is  PRECTt-dt
These time steps are cut: []
01/10/2022 04:19:17 PM S

OMP: Info #250: KMP_AFFINITY: pid 272716 tid 273152 thread 36 bound to OS proc set 25
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 273430 thread 37 bound to OS proc set 29
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 273431 thread 38 bound to OS proc set 33
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 273154 thread 39 bound to OS proc set 35
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 273437 thread 40 bound to OS proc set 31
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 273438 thread 41 bound to OS proc set 27
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 273156 thread 42 bound to OS proc set 37
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 273447 thread 43 bound to OS proc set 41
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 273448 thread 44 bound to OS proc set 45
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 273158 thread 45 bound to OS proc set 47
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 273454 thread 46 bound to OS proc set 43
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 273455 th

OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275491 thread 139 bound to OS proc set 41
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275490 thread 138 bound to OS proc set 37
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275492 thread 140 bound to OS proc set 45
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275493 thread 141 bound to OS proc set 47
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275494 thread 142 bound to OS proc set 43
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275495 thread 143 bound to OS proc set 39
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275496 thread 144 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275497 thread 145 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275498 thread 146 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275499 thread 147 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275500 thread 148 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 27

OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275672 thread 243 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275674 thread 245 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275673 thread 244 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275675 thread 246 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275676 thread 247 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275677 thread 248 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275678 thread 249 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275680 thread 251 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275679 thread 250 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275681 thread 252 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275682 thread 253 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 27

OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275860 thread 350 bound to OS proc set 9
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275859 thread 349 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275858 thread 348 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275861 thread 351 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275862 thread 352 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275864 thread 354 bound to OS proc set 13
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275863 thread 353 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275866 thread 356 bound to OS proc set 21
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275865 thread 355 bound to OS proc set 17
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275867 thread 357 bound to OS proc set 23
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275868 thread 358 bound to OS proc set 19
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275

...

OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276161 thread 558 bound to OS proc set 36
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276162 thread 559 bound to OS proc set 40
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276163 thread 560 bound to OS proc set 44
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276164 thread 561 bound to OS proc set 46
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276165 thread 562 bound to OS proc set 42
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276166 thread 563 bound to OS proc set 38
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276167 thread 564 bound to OS proc set 25
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276168 thread 565 bound to OS proc set 29
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276169 thread 566 bound to OS proc set 33
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276170 thread 567 bound to OS proc set 35
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276171 thread 568 bound to OS proc set 31
OMP: Info #250: KMP_AFFINITY: pid 272716 ti

OMP: Info #250: KMP_AFFINITY: pid 272716 tid 273171 thread 672 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276930 thread 673 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276932 thread 675 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276931 thread 674 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276933 thread 676 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276934 thread 677 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276935 thread 678 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276936 thread 679 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276937 thread 680 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276938 thread 681 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276939 thread 682 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276

OMP: Info #250: KMP_AFFINITY: pid 272716 tid 273167 thread 768 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 1184 thread 769 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 1185 thread 770 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 1187 thread 772 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 1186 thread 771 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 1188 thread 773 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 1189 thread 774 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 1190 thread 775 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 1191 thread 776 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 1192 thread 777 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 1193 thread 778 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 1194 thread 779 bound t

OMP: Info #250: KMP_AFFINITY: pid 272716 tid 14654 thread 852 bound to OS proc set 25
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 14656 thread 854 bound to OS proc set 33
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 14655 thread 853 bound to OS proc set 29
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 14657 thread 855 bound to OS proc set 35
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 14658 thread 856 bound to OS proc set 31
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 14659 thread 857 bound to OS proc set 27
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 14660 thread 858 bound to OS proc set 37
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 14662 thread 860 bound to OS proc set 45
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 14663 thread 861 bound to OS proc set 47
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 14664 thread 862 bound to OS proc set 43
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 14661 thread 859 bound to OS proc set 41
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 14665 thr

OMP: Info #250: KMP_AFFINITY: pid 272716 tid 273145 thread 963 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 18827 thread 965 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 18826 thread 964 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 18828 thread 966 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 18829 thread 967 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 18830 thread 968 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 18832 thread 970 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 18831 thread 969 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 18833 thread 971 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 18834 thread 972 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 18835 thread 973 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 18836 thread

In [5]:
!python preprocessing-11132019.py -c pp_config/2021_09_02_Train_Valid_For_Nando.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
01/10/2022 06:30:39 PM Preprocess training dataset with real_geography flag set to False
01/10/2022 06:30:39 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_09_02_TRAIN_For_Nando.nc
01/10/2022 06:30:39 PM Reading input files
01/10/2022 06:30:39 PM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0001-0[1-4]-*-*.nc
01/10/2022 06:33:56 PM Crop levels
01/10/2022 06:33:56 PM Create stacked dataarray
01/10/2022 06:33:56 PM Real geography flag set to False
01/10/2022 06:33:56 PM Aquaplanet simulation detected
var is  QBP
var is  TBP
var is  VBP
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHFLX
var is  PHQ
var is  TPHYSTND
var is  FSNT
var is  FSNS
var is  FLNT
var is  FLNS
var is  PRECT
These time steps are cut: []
01/10/2022 06:33:56 PM Stack and reshape dataarray
01/10/2022 06:34:25 PM Save dataarray as /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_09

OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78598 thread 48 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78704 thread 52 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78701 thread 49 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78702 thread 50 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78703 thread 51 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78705 thread 53 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78706 thread 54 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78707 thread 55 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78708 thread 56 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78709 thread 57 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78710 thread 58 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78712 thread 60 bound to OS proc set 

OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78608 thread 147 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78868 thread 148 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78869 thread 149 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78610 thread 150 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78874 thread 152 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78873 thread 151 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78875 thread 153 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78876 thread 154 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78877 thread 155 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78878 thread 156 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78879 thread 157 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78880 thread 158 bound to

OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78620 thread 249 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78979 thread 251 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78978 thread 250 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78981 thread 253 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78980 thread 252 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78982 thread 254 bound to OS proc set 9
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78984 thread 256 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78983 thread 255 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78985 thread 257 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78986 thread 258 bound to OS proc set 13
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78988 thread 260 bound to OS proc set 21
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78987 thread 259 bound to 

OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78630 thread 348 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79153 thread 350 bound to OS proc set 9
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79152 thread 349 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79155 thread 352 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79154 thread 351 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79156 thread 353 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79157 thread 354 bound to OS proc set 13
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79158 thread 355 bound to OS proc set 17
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79160 thread 357 bound to OS proc set 23
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79159 thread 356 bound to OS proc set 21
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79162 thread 359 bound to OS proc set 15
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79161 thread 358 bound to 

OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78640 thread 447 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79257 thread 448 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79258 thread 449 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78599 thread 450 bound to OS proc set 13
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79321 thread 451 bound to OS proc set 17
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79322 thread 452 bound to OS proc set 21
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79325 thread 455 bound to OS proc set 15
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79323 thread 453 bound to OS proc set 23
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79324 thread 454 bound to OS proc set 19
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79326 thread 456 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79327 thread 457 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79329 thread 459 bound 

01/10/2022 06:48:25 PM Done!
01/10/2022 06:48:26 PM Shuffle training dataset
01/10/2022 06:48:26 PM Start shuffling /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_09_02_TRAIN_For_Nando.nc into /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_09_02_TRAIN_For_Nando_shuffle.nc. Open and create datasets.
01/10/2022 06:48:28 PM Shuffle!
100%|█████████████████████████████████████████████| 5/5 [02:00<00:00, 23.84s/it]
01/10/2022 06:50:28 PM Closing datasets
01/10/2022 06:50:28 PM Done!
01/10/2022 06:50:28 PM Preprocess validation dataset
01/10/2022 06:50:28 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_09_02_VALID_For_Nando.nc
01/10/2022 06:50:28 PM Reading input files
01/10/2022 06:50:28 PM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0001-0[5-8]-*-*.nc
01/10/2022 06:54:01 PM Crop levels
01/10/2022 06:54:01 PM Create stacked dataarray
01/10/2022 06:54:01 PM Real geography flag set t

OMP: Info #250: KMP_AFFINITY: pid 72462 tid 91561 thread 639 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 91563 thread 641 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 91562 thread 640 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 91564 thread 642 bound to OS proc set 13
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 91565 thread 643 bound to OS proc set 17
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 91567 thread 645 bound to OS proc set 23
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 91566 thread 644 bound to OS proc set 21
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 91569 thread 647 bound to OS proc set 15
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 91568 thread 646 bound to OS proc set 19
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 91570 thread 648 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 91571 thread 649 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 91572 thread 650 bound 

OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78621 thread 744 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 93370 thread 745 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 93371 thread 746 bound to OS proc set 32
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 93424 thread 747 bound to OS proc set 34
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 93425 thread 748 bound to OS proc set 30
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 93426 thread 749 bound to OS proc set 26
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 93427 thread 750 bound to OS proc set 36
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 93428 thread 751 bound to OS proc set 40
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 93429 thread 752 bound to OS proc set 44
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 93430 thread 753 bound to OS proc set 46
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 93431 thread 754 bound to OS proc set 42
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 93432 thread 755 boun

In [3]:
!python preprocessing-11132019.py -c pp_config/2021_11_30_Norm_For_Nando_Cl_Inv.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
11/30/2021 07:58:34 AM Preprocess training dataset with real_geography flag set to False
11/30/2021 07:58:34 AM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_11_30_TRAIN_For_Nando_ClInv.nc
11/30/2021 07:58:34 AM Reading input files
11/30/2021 07:58:34 AM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0001-01-*-*.nc
11/30/2021 07:58:39 AM Crop levels
11/30/2021 07:58:39 AM Create stacked dataarray
11/30/2021 07:58:39 AM Real geography flag set to False
11/30/2021 07:58:39 AM Aquaplanet simulation detected
var is  RH
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0-47
OMP: Info #156: KMP_AFFINITY: 48 available OS procs
OMP: Info #157: KMP_AFFINITY: Uniform topology
OMP: Info #179: KMP_AFFINITY: 2 packages x 12 cores

OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114307 thread 48 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114470 thread 49 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114471 thread 50 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114472 thread 51 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114474 thread 53 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114473 thread 52 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114475 thread 54 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114476 thread 55 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114477 thread 56 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114478 thread 57 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114479 thread 58 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114480 thread 

OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114341 thread 144 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114609 thread 145 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114610 thread 146 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114611 thread 147 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114612 thread 148 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114613 thread 149 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114614 thread 150 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114615 thread 151 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114616 thread 152 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114617 thread 153 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114618 thread 154 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114

OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114348 thread 240 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114708 thread 242 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114707 thread 241 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114709 thread 243 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114710 thread 244 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114711 thread 245 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114712 thread 246 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114714 thread 248 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114713 thread 247 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114715 thread 249 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114716 thread 250 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114

OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114316 thread 336 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114805 thread 338 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114804 thread 337 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114806 thread 339 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114807 thread 340 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114808 thread 341 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114809 thread 342 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114810 thread 343 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114811 thread 344 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114813 thread 346 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114812 thread 345 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114

...

OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114323 thread 720 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115259 thread 721 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115260 thread 722 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115262 thread 724 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115261 thread 723 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115263 thread 725 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115264 thread 726 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115265 thread 727 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115266 thread 728 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115268 thread 730 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115269 thread 731 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115

OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114346 thread 816 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115357 thread 819 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115355 thread 817 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115356 thread 818 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115359 thread 821 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115360 thread 822 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115358 thread 820 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115362 thread 824 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115361 thread 823 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115363 thread 825 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115364 thread 826 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115

OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114310 thread 912 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115453 thread 913 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115454 thread 914 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115455 thread 915 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115456 thread 916 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115457 thread 917 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115459 thread 919 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115458 thread 918 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115460 thread 920 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115461 thread 921 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115462 thread 922 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115

OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114327 thread 1008 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115607 thread 1010 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115608 thread 1011 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115606 thread 1009 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115611 thread 1014 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115610 thread 1013 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115609 thread 1012 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115613 thread 1016 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115612 thread 1015 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115614 thread 1017 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115615 thread 1018 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 114

OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114338 thread 1104 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115712 thread 1105 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115713 thread 1106 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115715 thread 1108 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115714 thread 1107 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115716 thread 1109 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115719 thread 1112 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115720 thread 1113 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115718 thread 1111 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115717 thread 1110 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115722 thread 1115 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 114

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_09_02_Train_Valid_For_Nando.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_09_02_Test_for_Nando.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_09_03_Valid_for_Nando_Cl_Inv.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_09_03_Test_for_Nando_Cl_Inv.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_09_13_Nando_ClInv_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_06_16_T_BMSE_input_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_06_06_RG_Test-0K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_06_03_RG_Train_Valid-0K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_24_RG_PERC_Train_Valid-M4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_24_RG_PERC_Train_Valid-P4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_24_RG_PERC_Test-P4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_24_RG_PERC_Test-M4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_09_PERC_Test-P4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_09_PERC_Test-M4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_09_PERC_Train_Valid-P4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_09_PERC_Train_Valid.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_09_PERC_Test.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_03_31_O3_T_NSto220_input_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_03_18_O3_Train_Valid-P4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_03_18_O3_Test-P4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_03_18_O3_Train_Valid-M4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_03_18_O3_Test-M4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_02_01_O3_BCONS_input_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_03_01_O3_LHFnsDELQ_input_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_03_01_O3_LHFnsQ_input_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_03_01_O3_LHFnsDELQ_input_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_03_01_O3_LHFnsQ_input_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_02_01_O3_QSATdeficit_input_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_01_24_O3_input_small_data_test

In [ ]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_01_24_O3_Train_Valid.yml

In [ ]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_01_24_O3_Test.yml

In [ ]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/147_POG_RHinput_FLUXoutput_test.yml

In [ ]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/118_POG_TfromTSinput_test.yml

In [ ]:
!ls -la -h /local/Tom.Beucler/SPCAM_PHYS/1*

# 2) Check the preprocessed data

In [ ]:
path_data = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/'

In [ ]:
path_RGsmall = path_data + '2021_04_18_NORM_RG_small.nc'

In [ ]:
path_QSATdeficit = path_data+'2021_02_01_NORM_O3_QSATdeficit_small.nc'

In [ ]:
path_TfromNS = path_data+'2021_02_01_NORM_O3_TfromNS_small.nc'

In [ ]:
path_Bcons = path_data+'2021_03_17_NORM_O3_BCONS_small.nc'

In [ ]:
path_BMSE = path_data + '2021_06_16_NORM_BMSE_small.nc'

In [ ]:
data_RGsmall = xr.open_dataset(path_RGsmall)

In [ ]:
# data_QSATdeficit = xr.open_dataset(path_QSATdeficit)
# data_TfromNS = xr.open_dataset(path_TfromNS)
# data_Bcons = xr.open_dataset(path_Bcons)
data_Bmse = xr.open_dataset(path_BMSE)

In [ ]:
data_QSATdeficit

In [ ]:
data_TfromNS

In [ ]:
data_Bmse['var_names']

In [ ]:
data_TfromNS['mean'].shape

In [ ]:
data_Bmse['mean'][:30].values

In [ ]:
data_Bmse['mean'][30:60].values

In [ ]:
plt.hist(data_Bmse['mean'])

In [ ]:
data_Bcons['mean'][30:60]

In [ ]:
data_Bcons['std'][30:60]

# 3) Preprocessing for PRL paper

In [ ]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/8col_rad_tbeucler_local-RH-test.yml

In [ ]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/8col_rad_tbeucler_local-test.yml